In [ ]:
#| hide
#| eval: false

@dataclass
class SegmentEmission:
    mode: Route_Mode
    distance_m: float
    co2e_g: float

@dataclass
class TripEmissionEstimate:
    segments: List[SegmentEmission]
    total_distance_m: float
    total_co2e_g: float

_EMISSION_FACTORS: dict[str, float] = {
    "car" : 171.0,
    "bike": 21.0,
    "walk": 56.0,
    "pt"  :  65.0,   # typical urban-public-transport mix (bus+rail)
}

#@mcp.tool(
#    name="estimate_trip_emissions",
#    description=(
#        "Given a list of Route objects that form a multimodal trip, return "
#        "segment-by-segment and total CO₂-e estimates using average factors."
#    ),
#)
def estimate_trip_emissions(routes: List[Route]) -> TripEmissionEstimate:
    segs: List[SegmentEmission] = []
    for rt in routes:
        factor = _EMISSION_FACTORS.get(rt.mode, 0.0) 
        co2_g  = factor * (rt.distance_m / 1000)
        segs.append(SegmentEmission(rt.mode, rt.distance_m, co2_g))

    total_d = sum(s.distance_m for s in segs)
    total_c = sum(s.co2e_g     for s in segs)

    return TripEmissionEstimate(
        segments=segs,
        total_distance_m=total_d,
        total_co2e_g=total_c,
    )

In [ ]:
#| hide
#| eval: false

@app.on_event("startup") 
async def _auto_mount_digitraffic() -> None:
    try:
        tools = await _mount_digitraffic_api(mcp, name="digitraffic")
        logging.info("✅ Digitraffic API mounted with %s tools: %s", len(tools), tools)
    except Exception as exc:
        logging.error("❌ Could not mount Digitraffic API: %s", exc)

In [ ]:
#| hide
#| eval: false
"""OSRM routes, strange results. Outdated?"""

@mcp.tool(
    name="car_route",
    description=(
        "Compute the fastest **car / driving route** with the Open-Source-Routing-Machine "
        "HTTP *Route* service (`/route/v1/driving`) on the public demo host "
        "`https://router.project-osrm.org` (OSRM v6.0, April 2025). \n\n"
        "Inputs\n"
        "------\n"
        "• `start_point`  – address, POI, or \"lat,lon\" string (first geocoded).\n"
        "• `finish_point` – same format.\n\n"
        "Behaviour\n"
        "---------\n"
        "• Calls OSRM with defaults `overview=full`, `geometries=polyline6`, "
        "`annotations=distance,duration`. Picks **routes[0]** (quickest) only.\n"
        "• Profile fixed to `driving` ⇒ speed model ≈ legal speed limits, "
        "traffic *not* considered.\n"
        "• **Public demo has a fair-use cap (~5 000 req/min shared)** — heavy use "
        "may return HTTP 429 or 400 :contentReference[oaicite:0]{index=0}.\n\n"
        "Returns (`str` → JSON Route)\n"
        "---------------------------\n"
        "`{\n"
        "  code, waypoints[],\n"
        "  route: {\n"
        "    distance_m, duration_s, weight, geometry (polyline6),\n"
        "    legs[], steps[]\n"
        "  }\n"
        "}` — ready for Leaflet-polyline or CO₂ post-processing.\n\n"
        "Use when the user explicitly wants to drive or asks for a car comparison."
    ),
)
async def car_route(
    start_point: str,
    finish_point: str,
) -> str:
    """Car route via public OSRM demo."""
    start_point = await geocode(start_point)
    finish_point = await geocode(finish_point)
    return await osrm_car(start_point, finish_point)


@mcp.tool(
    name="walk_route",
    description=(
        "Return a **pedestrian itinerary** with OSRM `/route/v1/foot` (foot.lua profile) "
        "on the public demo server. Uses the new *pedestrian-platform* support "
        "introduced in **OSRM v6.0** :contentReference[oaicite:1]{index=1}.\n\n"
        "Inputs: identical to `car_route` (start/finish geocoded).\n\n"
        "Behaviour\n"
        "---------\n"
        "• Walking speed ≈ 4.8 km/h; avoids motorways & trunks by default.\n"
        "• Returns only the primary route; geometry encoded as *polyline6*.\n\n"
    ),
)
async def walk_route(
    start_point: str,
    finish_point: str,
) -> str:
    "Walking route via OSRM."
    start_point = await geocode(start_point)
    finish_point = await geocode(finish_point)
    return await osrm_walk(start_point, finish_point)


@mcp.tool(
    name="bike_route",
    description=(
        "Generate a **cycling route** with OSRM `/route/v1/bicycle` (bicycle.lua profile, "
        "public demo). Suitable for conventional and e-bikes; max speed ≈ 18 km/h. \n\n"
        "Inputs: two free-form locations (geocoded).\n\n"
        "Behaviour\n"
        "---------\n"
        "• Prefers `highway=cycleway|living_street` where available, respects OSM "
        "`bicycle=no` restrictions.\n"
        "• Emits first route only, with `geometry=polyline6`, `steps=true` so the "
        "agent can show turn-by-turn cues.\n"
    ),
)
async def bike_route(
    start_point: str,
    finish_point: str,
) -> str:
    start_point = await geocode(start_point)
    finish_point = await geocode(finish_point)
    """Walking route via OSRM."""
    return await osrm_bike(start_point, finish_point)

@mcp.tool(
    name="public_transport_route",
    description=(
        "Return the **fastest door-to-door public-transport itinerary** between two user-supplied "
        "locations in the Finnish HSL region, powered by **Digitransit Routing API v2** "
        "(GraphQL `planConnection`, endpoint `/routing/v2/hsl/gtfs/v1`).\n\n"
        "Inputs\n"
        "------\n"
        "• `start_point`  – free-form stop name, address, POI or \"lat,lon\" string.\n"
        "• `finish_point` – same format as above.\n"
        "(Both are first geocoded with Digitransit’s Geocoding API, then rounded to 6 decimals.)\n\n"
        "Behaviour\n"
        "---------\n"
        "• Queries with defaults `{ walkSpeed: 4.8 km/h, maxTransfers: 3 }` and picks the **first** "
        "(i.e. quickest) itinerary returned by Digitransit.\n"
        "• Dataset is fixed to **`hsl`** – Helsinki Region Transport.\n\n"
       
    )
)
async def public_transport_route(
    start_point: str,
    finish_point: str,
) -> str:
    """Fastest public-transport itinerary via Digitransit Routing v2."""
    start_point = await geocode(start_point)
    finish_point = await geocode(finish_point)
    return await digitransit_pt(start_point, finish_point)


#@mcp.tool()
async def geocode_osm(query: str) -> dict:
    """
    Best-effort geocoder backed by the public **Nominatim** service.

    What this does
    --------------
    • Sends the exact text you pass as `q=<query>` to
      https://nominatim.openstreetmap.org/search?… and returns the first hit
      as `{"lat": <float>, "lon": <float>}`. :contentReference[oaicite:0]{index=0}  
    • If Nominatim returns an empty list we raise **ValueError** – the agent
      will see this as a tool failure. :contentReference[oaicite:1]{index=1}  

    Nominatim quirks you *must* know
    --------------------------------
    1. **AND search** – every whitespace-separated token must appear in the
       OSM object’s name/aliases; it never “drops” words automatically. :contentReference[oaicite:2]{index=2}  
    2. Finnish metro objects are simply `name=Vuosaari`, `name=Urheilupuisto`
       … The words “metro” / “station” **are not in the tags**, so  
       `q="Vuosaari metro station"` → 0 results → ValueError. :contentReference[oaicite:3]{index=3}  
    3. The public server is throttled to **1 request per second per IP**;
       abuse will get you 429s. :contentReference[oaicite:4]{index=4}  

    Crafting a good query
    ---------------------
    ✔ Pass **just the place name first**:  
      `"Vuosaari"`, `"Urheilupuisto"`, `"Kamppi"` …  
    ✔ Add disambiguators *after* a comma if needed:  
      `"Turku, Finland"` → keeps both tokens but lets Nominatim match
      “Turku” freely. :contentReference[oaicite:5]{index=5}  
    ✔ Prefer structured filters instead of extra words:  
      • `countrycodes=fi` narrows to Finland :contentReference[oaicite:6]{index=6}  
      • `viewbox=<xmin>,<ymin>,<xmax>,<ymax>&bounded=1` restricts to a bbox :contentReference[oaicite:7]{index=7}  

    What to do when it still fails
    ------------------------------
    ① Strip generic suffixes and retry once (“metro station”, “asema”,  
       “train station”).  
    ② If that also fails **and** the token “metro” is present, fall back to
       the HSL **Digitransit GraphQL** geocoder – it indexes every stop &
       entrance in the Helsinki region. Example:  

       ```graphql
       { stops(name:"Vuosaari") { gtfsId name lat lon } }
       ``` :contentReference[oaicite:8]{index=8}  

    The wrapper already implements ①; callers only need to `await` it.  
    For ②, catch **ToolError** and run a Digitransit query if you need
    rock-solid results.

    Example
    -------
    ```python
     await geocode_osm("Vuosaari")
    {'lat': 60.2082539, 'lon': 25.1564019}

    await geocode_osm("Vuosaari metro station")
    ValueError: No location found for 'Vuosaari metro station'
    ```

    Performance tips
    ----------------
    • Wrap this call in an LRU or aio-cache to avoid hitting the 1 req/s cap.  
    • Re-use results inside a chat session; users almost always
      mention the same place more than once.

    Returns
    -------
    dict
        Keys: ``lat`` and ``lon`` as float.

    Raises
    ------
    ValueError
        When Nominatim returns no result or the HTTP request fails.
    """
    try:
        return (await geocode(query)).model_dump()
    except LookupError as e:
        # surface a predictable ToolError so the agent can fall back
        raise ValueError(str(e))


In [ ]:
#| hide
#| eval: false

import os
import json
import logging
import httpx
from pathlib import Path
from fastmcp import FastMCP
from fastmcp.server.openapi import RouteMap, MCPType

_DIGI_SPEC_URL   = "https://tie.digitraffic.fi/swagger/openapi.json"
_DIGI_SPEC_PATH  = "digitraffic_openapi.json"
_BASE_URL        = "https://tie.digitraffic.fi"
_USER_HEADER     = os.getenv("DIGITRAFFIC_USER", "mycompany/myapp 1.0")

async def _mount_digitraffic_api(parent: FastMCP,
                                 name: str = "digitraffic") -> list[str]:
    # 1. Configure HTTPX client with required headers
    client = httpx.AsyncClient(
        base_url=_BASE_URL,
        headers={
            "Digitraffic-User": _USER_HEADER,  # recommended by Fintraffic :contentReference[oaicite:10]{index=10}
            "Accept-Encoding": "gzip",         # mandatory per API guide :contentReference[oaicite:11]{index=11}
        },
        timeout=30.0,
    )

    # 2. Load (or download) the OpenAPI spec
    spec_path = Path(_DIGI_SPEC_PATH)
    if not spec_path.exists():
        resp = await client.get(_DIGI_SPEC_URL)
        resp.raise_for_status()
        spec = resp.json()
        spec_path.write_text(json.dumps(spec, indent=2), encoding="utf-8")
    else:
        spec = json.loads(spec_path.read_text(encoding="utf-8"))

    # 3. Keep only the traffic-message endpoints
    route_maps = [
        RouteMap(
            pattern=r"^/api/traffic-message/v1/messages$",
            methods=["GET"],
            mcp_type=MCPType.TOOL
        ),
        RouteMap(
            pattern=r"^/api/traffic-message/v1/messages\\.datex2$",
            methods=["GET"],
            mcp_type=MCPType.TOOL
        ),
        RouteMap(mcp_type=MCPType.EXCLUDE),
    ]

    sub = FastMCP.from_openapi(
        openapi_spec=spec,
        client=client,
        name="digitraffic",
        route_maps=route_maps
    )

    parent.mount(name, sub)
    


    return [t.name for t in (await sub.get_tools()).values()]


